In [165]:
library(data.table)

In [182]:
feature_dt_hour=fread("C:/Users/bahad/GitHub/360project/hourly_feature_df.csv")
feature_dt_hour=feature_dt_hour[,-1]
feature_dt_hour$date=as.Date(feature_dt_hour$date)

In [183]:
req_feature_hour=feature_dt_hour[date>"2020-06-06"&date<=Sys.Date()-8]

In [184]:
str(feature_dt_hour)

Classes 'data.table' and 'data.frame':	35832 obs. of  9 variables:
 $ date                    : Date, format: "2017-01-01" "2017-01-01" ...
 $ is_lockdown             : int  0 0 0 0 0 0 0 0 0 0 ...
 $ is_sun                  : int  0 0 0 0 0 0 0 0 0 1 ...
 $ holiday_val_factor      : num  1 1 1 1 1 1 1 1 1 1 ...
 $ weekday_factor          : num  7 7 7 7 7 7 7 7 7 7 ...
 $ covid_severity          : num  0 0 0 0 0 0 0 0 0 0 ...
 $ production_capacity_rate: num  77 77 77 77 77 77 77 77 77 77 ...
 $ price_of_electricity    : num  20.5 20.5 20.5 20.5 20.5 ...
 $ usdtry                  : num  3.53 3.53 3.53 3.53 3.53 ...
 - attr(*, ".internal.selfref")=<externalptr> 


In [185]:
time_cons1=read.csv("bulk_consumption_with_temp.csv")
time_cons2=read.csv("current_data.csv")
time_cons2=time_cons2[,-1]

In [186]:
time_cons=rbind(time_cons1,time_cons2)
setnames(time_cons,"Date","date")
time_cons=as.data.table(time_cons)
time_cons$date=as.Date(time_cons$date)

In [187]:
str(time_cons)

Classes 'data.table' and 'data.frame':	35832 obs. of  10 variables:
 $ date       : Date, format: "2017-01-01" "2017-01-01" ...
 $ Hour       : int  0 1 2 3 4 5 6 7 8 9 ...
 $ Consumption: num  25409 24166 22652 21578 21018 ...
 $ T_1        : num  -15.9 -15.9 -15.9 -15.9 -15.9 ...
 $ T_2        : num  4.18 4.18 4.18 4.18 4.18 4.18 4.18 4.33 4.48 4.63 ...
 $ T_3        : num  0.89 0.89 0.89 0.89 0.89 0.89 0.89 0.92 0.95 0.98 ...
 $ T_4        : num  -19 -19 -19 -19 -19 ...
 $ T_5        : num  -14.8 -14.8 -14.8 -14.8 -14.8 ...
 $ T_6        : num  -10.7 -10.7 -10.7 -10.7 -10.7 ...
 $ T_7        : num  2.16 2.16 2.16 2.16 2.16 2.16 2.16 1.82 1.48 1.14 ...
 - attr(*, ".internal.selfref")=<externalptr> 


In [188]:
time_cons[,cons_daily:= sum(Consumption) ,by=date]
time_cons[,perc:= Consumption/cons_daily]

In [189]:
for(i in 0:23) {
  assign(paste0("hour",i), time_cons[Hour==i & date>"2020-06-06"&date<=Sys.Date()-8,])
  }

In [190]:
for(i in 0:23){
    #plot(get(paste0("hour",i))[date>"2020-06-01"&date<=Sys.Date()-8]$perc,type="l")
    assign(paste0("decompose",i),decompose(ts(get(paste0("hour",i))$perc,freq=7),
                                           type='add'))
    #plot(get(paste0("decompose",i)))
}

In [191]:
req_feature_hour$hour=rep(0:23,nrow(req_feature_hour)/24)

# Hour

In [192]:
library(forecast)

In [193]:
res_hour_per=data.table(hour=0:23)

In [214]:
results_percentage=vector("list",24)

for(i in 0:23){
    new_forescasted=0
    fit_mod=auto.arima(get(paste0("decompose",i))$random)
    new_forescasted=as.numeric(predict(fit_mod, n.ahead = 1)$pred +
                               get(paste0("decompose",i))$season[wday(Sys.Date())] +
                                     tail(get(paste0("decompose",i))$trend[!is.na(get(paste0("decompose",i))$trend)],1))
    results_percentage[[i+1]]=new_forescasted
}    



In [215]:
Percentage_dt=t(rbind(results_percentage))